In [2]:
import pickle
import pandas as pd
import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

import warnings
warnings.filterwarnings("ignore")

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
scores = dict()
df = pd.read_csv('input/train.csv')

print('data loaded...')

In [58]:
class UbiquantData(Dataset):
    def __init__(self, data: pd.core.frame.DataFrame):
        self.target = data[['target']].values
        self.data = data.drop(['row_id', 'time_id', 'investment_id', 'target'], axis=1).values

    def __getitem__(self, idx):
        x_cont = self.data[idx]
        target = self.target[idx]
        return torch.tensor(x_cont).float(), torch.tensor(target).float()

    def __len__(self):
        return len(self.data)

In [60]:
epochs = 20
l_rate = 1e-3
mse_loss = nn.MSELoss()

def swish(x):
    return x * torch.sigmoid(x)

def weighted_average(a):
    w = []
    n = len(a)
    for j in range(1, n + 1):
        j = 2 if j == 1 else j
        w.append(1 / (2**(n + 1 - j)))
    return np.average(a, weights = w)

class UbiquantModel(pl.LightningModule):
    def __init__(self):
        super(UbiquantModel, self).__init__()
        self.bn = nn.BatchNorm1d(300)
        self.fc1 = nn.Linear(300, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.bn(x)
        x = swish(self.fc1(x))
        x = swish(self.fc2(x))
        x = swish(self.fc3(x))
        x = swish(self.fc4(x))
        x = self.fc5(x)
        return x

    def train_dataloader(self):
        train_dataset = UbiquantData(train_data)
        train_loader = DataLoader(dataset=train_dataset, batch_size=4096)
        return train_loader

    def val_dataloader(self):
        val_dataset = UbiquantData(val_data)
        validation_loader = DataLoader(dataset=val_dataset, batch_size=4096)
        return validation_loader

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=l_rate)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = mse_loss(logits, y)
        logs = {'loss': loss}
        return {'loss': loss, 'log': logs}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        scores_df = pd.DataFrame(index=range(len(y)), columns=['targets', 'preds'])
        logits = self.forward(x)
        scores_df.targets = y.cpu().numpy()
        scores_df.preds = logits.cpu().numpy()
        pearson = scores_df['targets'].corr(scores_df['preds'])
        pearson = np.array(pearson)
        pearson = torch.from_numpy(pearson)
        self.log('pearson', pearson)
        return {'val_loss': pearson}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        print('mean pearson correlation on validation set: ', avg_loss)
        if fold not in scores:
            scores[fold] = [avg_loss]
        else:
            scores[fold].append(avg_loss)
        tensorboard_logs = {'val_loss': avg_loss}
        return {'avg_val_loss': avg_loss, 'log': tensorboard_logs}

In [ ]:
scores = dict()

with open('input/folds.pickle', 'rb') as handle:
    fold_indexes = pickle.load(handle)

for fold in fold_indexes:
    remove_fields = ['target', 'row_id', 'time_id', 'investment_id']
    target_fields = ['target']

    train_data = df.iloc[fold_indexes[fold]['train']]
    validation_data = df.iloc[fold_indexes[fold]['test']]
    
    checkpoint_callback = ModelCheckpoint(
        monitor="pearson",
        dirpath="models",
        filename="fold-" + str(fold) + "-ubiquant-mlp-{epoch:02d}-{val_loss:.2f}",
        save_top_k=1,
        mode="max",
    )

    model = UbiquantModel()
    trainer = Trainer(max_epochs=epochs,
                      fast_dev_run=False,
                      callbacks=[checkpoint_callback],
                      gpus=1)
    trainer.fit(model)

score_list = list()
for fold in scores:
    score_list.append(max(scores[fold]))

print('final weighted correlation for the experiment: ', weighted_average(score_list))

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type        | Params
-------------------------------------
0 | bn   | BatchNorm1d | 600   
1 | fc1  | Linear      | 154 K 
2 | fc2  | Linear      | 65.7 K
3 | fc3  | Linear      | 8.3 K 
4 | fc4  | Linear      | 2.1 K 
5 | fc5  | Linear      | 33    
-------------------------------------
230 K     Trainable params
0         Non-trainable params
230 K     Total params
0.923     Total estimated model params size (MB)


Epoch 0:  50%|█████████████████████████▋                         | 130/258 [00:11<00:11, 11.13it/s, loss=1.02, v_num=44]
Validating: 0it [00:00, ?it/s]
Epoch 1:  50%|█████████████████████████▋                         | 130/258 [00:11<00:11, 11.29it/s, loss=1.01, v_num=44]
Validating: 0it [00:00, ?it/s]
Epoch 2:  50%|█████████████████████████▏                        | 130/258 [00:10<00:10, 12.11it/s, loss=0.999, v_num=44]
Validating: 0it [00:00, ?it/s]
Epoch 3:  48%|████████████████████████                          | 124/258 [00:10<00:11, 11.80it/s, loss=0.963, v_num=44]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type        | Params
-------------------------------------
0 | bn   | BatchNorm1d | 600   
1 | fc1  | Linear      | 154 K 
2 | fc2  | Linear      | 65.7 K
3 | fc3  | Linear      | 8.3 K 
4 | fc4  | Linear      | 2.1 K 
5 | fc5  | Linear      | 33    
-------------------------------------
230 K     Trainable params
0         Non-trainable params
230 K     Total params
0.923     Total estimated model params size (MB)


Epoch 0:  67%|█████████████████████████████████▍                | 258/386 [00:23<00:11, 10.77it/s, loss=0.998, v_num=45]
Validating: 0it [00:00, ?it/s]
Epoch 1:  67%|█████████████████████████████████▍                | 258/386 [00:23<00:11, 11.02it/s, loss=0.989, v_num=45]
Validating: 0it [00:00, ?it/s]
Epoch 2:  67%|█████████████████████████████████▍                | 258/386 [00:22<00:11, 11.27it/s, loss=0.975, v_num=45]
Validating: 0it [00:00, ?it/s]
Epoch 3:  67%|█████████████████████████████████▍                | 258/386 [00:22<00:11, 11.25it/s, loss=0.956, v_num=45]
Validating: 0it [00:00, ?it/s]
Epoch 4:  67%|█████████████████████████████████▍                | 258/386 [00:24<00:12, 10.53it/s, loss=0.932, v_num=45]
Validating: 0it [00:00, ?it/s]
Epoch 5:  67%|█████████████████████████████████▍                | 258/386 [00:23<00:11, 10.99it/s, loss=0.911, v_num=45]
Validating: 0it [00:00, ?it/s]
Epoch 6:  67%|█████████████████████████████████▍                | 258/386 [00:24<00:12, 

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type        | Params
-------------------------------------
0 | bn   | BatchNorm1d | 600   
1 | fc1  | Linear      | 154 K 
2 | fc2  | Linear      | 65.7 K
3 | fc3  | Linear      | 8.3 K 
4 | fc4  | Linear      | 2.1 K 
5 | fc5  | Linear      | 33    
-------------------------------------
230 K     Trainable params
0         Non-trainable params
230 K     Total params
0.923     Total estimated model params size (MB)


Epoch 0:  75%|█████████████████████████████████████▌            | 385/513 [00:32<00:10, 11.90it/s, loss=0.941, v_num=47]
Validating: 0it [00:00, ?it/s]
Epoch 1:  75%|█████████████████████████████████████▌            | 385/513 [00:32<00:10, 11.69it/s, loss=0.933, v_num=47]
Validating: 0it [00:00, ?it/s]
Epoch 2:  75%|█████████████████████████████████████▌            | 385/513 [00:32<00:10, 11.76it/s, loss=0.924, v_num=47]
Validating: 0it [00:00, ?it/s]
Epoch 3:  75%|█████████████████████████████████████▌            | 385/513 [00:33<00:11, 11.51it/s, loss=0.913, v_num=47]
Validating: 0it [00:00, ?it/s]
Epoch 4:  75%|███████████████████████████████████████             | 385/513 [00:33<00:11, 11.57it/s, loss=0.9, v_num=47]
Validating: 0it [00:00, ?it/s]
Epoch 5:  75%|█████████████████████████████████████▌            | 385/513 [00:32<00:10, 11.74it/s, loss=0.887, v_num=47]
Validating: 0it [00:00, ?it/s]
Epoch 6:  75%|█████████████████████████████████████▌            | 385/513 [00:32<00:10, 